## Setup

### Imports

In [ ]:
import sys; sys.path.append('..')

In [ ]:
# ! pip install lovely-tensors

import lovely_tensors as lt
lt.monkey_patch()

In [ ]:
import torch
import numpy as np
import torch.nn.functional as F
from torch.utils.data import DataLoader

from transformer_lens import HookedTransformer, HookedTransformerConfig

In [ ]:
import wandb
from tqdm.auto import tqdm

from omegaconf import OmegaConf

from datetime import datetime

from pathlib import Path

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from src.tree import list2tree
from src.tree_dataset import TreeDataset, parse_input_idx, input_tokens_to_tree, tree_to_edges
from src.utils import seed_all, count_parameters


from src.trainer import TreeTrainer

In [ ]:
from src.tree_dataset import TreeDataset, DeepTreeDataset, parse_input_idx, input_tokens_to_tree, tree_to_edges

In [ ]:
import pandas as pd

@torch.inference_mode()
def get_test_metrics(n_test_batches):
    metric_df = []
    
    for i, batch in tqdm(zip(range(n_test_batches), test_dataloader), total=n_test_batches):
        with torch.inference_mode():
            loss, metrics = trainer.train_step(batch)
            metric_df.append(metrics)
    
    
    metric_df = pd.DataFrame(metric_df)
    
    metric_dict = metric_df.mean().to_dict()
    metric_dict = {'test/'+k:v for k,v in metric_dict.items()}
    return metric_dict

### Read config

In [ ]:
conf = OmegaConf.load('../conf/01_2L_nodes=8__phase_trans.yaml')
conf = OmegaConf.load('../conf/00_01_reproduce_deep_distribution.yaml')

# Output is identical to the YAML file
print(OmegaConf.to_yaml(conf))

### Constants (mostly derived)

In [ ]:
DEV_RUN = False
USE_WANDB = (not DEV_RUN) and conf.use_wandb
device = conf.device

CHECKPOINT_ROOT = Path('../checkpoints')

In [ ]:
def create_checkpoint_dir(conf):
    now = datetime.now()
    now_filename = now.strftime("%Y-%m-%d_%H-%M-%S")
    
    checkpoint_dir = CHECKPOINT_ROOT/f'{conf["wandb"]["project"]}__{now_filename}'
    checkpoint_dir.mkdir(exist_ok=True, parents = True)
    return checkpoint_dir

In [ ]:
if USE_WANDB:
    wandb.init(project=conf.wandb.project, name=conf.wandb.name, config=OmegaConf.to_object(conf))
    print(OmegaConf.to_yaml(conf))

In [ ]:
RANDOM_SEED = conf['random_seed']
print(f'{RANDOM_SEED=}')
seed_all(RANDOM_SEED)

In [ ]:
trainer = TreeTrainer(conf)
count_parameters(trainer.model)

In [ ]:
tokenizer = trainer.dataset.tokenizer

tok = tokenizer.tokenize
detok = tokenizer.detokenize


ROOT_DELIM_TOKEN_IDX = trainer.tok([':'])[0]

In [ ]:
n_test_batches = 10
test_dataloader = DataLoader ( DeepTreeDataset(possible_depths=list(range(1,16))), batch_size=conf['batch_size'] )

In [ ]:
checkpoint_dir = create_checkpoint_dir(conf)

In [ ]:
global_step = 0
epoch_i = 0


while True:

    pbar = tqdm(list(range(conf.epoch_len_steps)))

    for i, batch in zip(pbar, trainer.dataloader):
        if i == 0:
            sample_input_idx = batch['input_idx'][0]
            trainer.print_sample_pred(sample_input_idx)
    
        trainer.optimizer.zero_grad()
        loss, metrics = trainer.train_step(batch)
        loss.backward()
        trainer.optimizer.step()
        
        # pbar.update()
        pbar.set_description(f'loss={float(loss):.3f}')
        if USE_WANDB:
            wandb.log({'loss': loss, **metrics})

        global_step += 1


        if global_step % 250 == 0:
            test_metrics = get_test_metrics(n_test_batches)
            wandb.log(test_metrics)

    epoch_i += 1

    if epoch_i % conf['checkpoint_every_epoch'] == 0:
        checkpoint_filename = checkpoint_dir/f'{conf["wandb"]["name"]}__step={global_step}.pt'
        print(f'Saving {checkpoint_filename=}')
        torch.save(trainer.model.state_dict(), checkpoint_filename)
    
    if conf.get('max_iters', False) and global_step > conf.get('max_iters'):
        break

    

In [ ]:
checkpoint_filename = checkpoint_dir/f'{conf["wandb"]["name"]}__step={global_step}.pt'
print(f'Saving {checkpoint_filename=}')
torch.save(trainer.model.state_dict(), checkpoint_filename)

### 

In [ ]:

import pandas as pd

import plotly.graph_objects as go

In [ ]:
def split_depths(mdic):
    return (int(k.split('=')[1]) for k in mdic.keys() if '=' in k)

metrics_reproduced = metric_df.mean().to_dict()

depths = sorted(set(split_depths(metrics_reproduced)))

reproduced_acc = [metrics_reproduced.get(f'acc/depth={d}', None) for d in depths]

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=depths, y=reproduced_acc, mode='lines+markers'))

fig.update_layout(title=f'Accuracy by Goal Depth. sample_size={n_test_batches*conf.batch_size}',
                   xaxis_title='Depth',
                   yaxis_title='Accuracy',
                   template='plotly_white')

fig.show()


In [ ]:
fig.write_html("plot.html")

In [ ]:


n_test_batches = 50

for batch in test_dataloader

## Debug on custom input

In [ ]:
for batch in trainer.dataloader:
    print(parse_input_idx(batch['input_idx'][0], trainer.dataset.tokenizer)['tree'])
    break

In [ ]:
with torch.inference_mode():
    locc, accuracy = trainer.train_step(batch)

In [ ]:
input_idx_custom = batch['input_idx'][0]


parsed_input = parse_input_idx(input_idx_custom, trainer.dataset.tokenizer)
parsed_input

In [ ]:
upper_task_bound = input_idx_custom.tolist().index(ROOT_DELIM_TOKEN_IDX) + 2
prompt_autoregressive = detok(input_idx_custom)[:upper_task_bound]
prompt = prompt_autoregressive
print(prompt)

In [ ]:
new_token = inference_on_prompt(prompt)
prompt += [new_token]
print(prompt)

In [ ]:
softmax_probs.shape

In [ ]:
softmax_probs = preds[0][mask[0]].softmax(dim=-1).detach().cpu().numpy()

# Plotting softmax probabilities as a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(softmax_probs, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Softmax Probabilities Heatmap")
plt.xlabel("Classes")
plt.ylabel("Instances")
plt.show()

In [ ]:
input_idx_autoregressive

In [ ]:
preds = trainer.model(input_idx)

In [ ]:
detok(input_idx[0][mask[0]])

In [ ]:
batch['input_idx'][:1]

In [ ]:
accuracy

In [ ]:




custom_tree = list2tree([0, 1, 2, None, None, 3])
custom_tree

In [ ]:
prompt_tokens, path_tokens = tree2tokens(custom_tree)
prompt_tokens

In [ ]:
self = trainer.dataset
prompt_idx = self.tokenizer(prompt_tokens)
path_idx = self.tokenizer(path_tokens)

input_tokens = prompt_idx + path_idx
pad_len = self.tokenizer.MAX_SEQ_LEN - len(input_tokens)

input_idx = torch.tensor(input_tokens + [0] * pad_len)
# pad_mask = torch.zeros(self.tokenizer.MAX_SEQ_LEN)
# pad_mask[:len(input_idx)] = 1

task_mask = torch.zeros(self.tokenizer.MAX_SEQ_LEN, dtype=torch.bool)
task_mask[len(prompt_idx):len(input_tokens)] = True

In [ ]:
trainer.model(input_idx).argmax(dim=-1)

In [ ]:
custom_input = ['0', ',', '→2', '0', ',', '→1', '1', ',', '→3', '|', '0', ':', '3', '→1', '→3']
tok(custom_input)

In [ ]:
custom_batch = {
                'input_idx': input_idx[None],
                'task_mask': task_mask[None],
                # 'pad_mask': pad_mask,
            }

In [ ]:
print_preds(custom_batch)

In [ ]:
input_idx = custom_batch['input_idx'].to(device)
mask = batch['task_mask'].to(device)

preds = trainer.model(custom_batch['input_idx'][:, :])

In [ ]:
detok(preds.argmax(dim=-1)[0])

In [ ]:
tree_to_edges(parsed_input['tree'])

In [ ]:
input_idx = batch['input_idx'].to(device)
mask = batch['task_mask'].to(device)

In [ ]:
logits, cache = trainer.model.run_with_cache(input_idx[:1])

In [ ]:
# pip install circuitsvis

In [ ]:
import circuitsvis as cv

In [ ]:
# ! pip install webbrowser

In [ ]:
# path = "attn_heads.html"

# with open(path, "w") as f:
#     f.write(str(attn_heads))

# webbrowser.open(path)


In [ ]:
parse_input_idx((input_idx[0]), tokenizer)[:0]

In [ ]:
print(type(cache))
attention_pattern = cache["pattern", 0]
print(attention_pattern.shape)
str_tokens = detok (input_idx[0].cpu())

print("Layer 0 Head Attention Patterns:")
attn_heads = (cv.attention.attention_patterns(
    tokens=str_tokens,
    attention=attention_pattern[0],
    attention_head_names=[f"L0H{i}" for i in range(1)],
))


attn_heads

In [ ]:
attention_pattern.shape

In [ ]:
%pip install git+https://github.com/callummcdougall/CircuitsVis.git#subdirectory=python

In [ ]:
path = "attn_heads.html"

with open(path, "w") as f:
    f.write(str(attn_heads))

In [ ]:
cv.__version__

In [ ]:
wandb.log({"custom_file": wandb.Html(open(path))})

In [ ]:
! python --version

In [ ]:
display(attn_heads)

In [ ]:
# Python Example
from circuitsvis.tokens import colored_tokens
colored_tokens(["My", "tokens"], [0.123, -0.226])